In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, Comment
import re
import time

In [2]:
url = 'https://www.pro-football-reference.com/teams'

In [3]:
#Get the urls of all the teams from the team landing page
def get_team_urls(main_url,year):
    #Get the page
    teams_page = requests.get(main_url)
    #Beautiful Soup the page
    teams_soup = BeautifulSoup(teams_page.content,"html.parser")
    #Use Beautiful Soup to find the active teams
    table_soup = teams_soup.find(id="teams_active")
    #Find all table headers (th) with class = "left"
    elements = table_soup.find_all("th",class_="left")
    team_list = []
    #Loop over all active teams
    for e in elements:
        #Get the team urls
        link = e.find("a")
        base_url = 'https://www.pro-football-reference.com'
        if link:
            team_list += [base_url + link["href"] + f"{year}_roster.htm"]
    return team_list

In [4]:
def get_player_urls(r_url):


    #Get the html for the team
    roster_page = requests.get(r_url)
    #Use Beautiful Soup on the HTML content
    roster_soup = BeautifulSoup(roster_page.content,"html.parser")
    #find the all_roster div
    roster_div = roster_soup.find(id="all_roster")
    #prettify the roster html 
    pretty_roster = roster_div.prettify()
    #Remove all the comments (The seem to add comments to make the scraping harder)
    roster_no_comments = pretty_roster.replace(" <!--","").replace("-->","")
    player_list = []

    #Use Beautiful soup on the uncommented html
    new_soup = BeautifulSoup(roster_no_comments,"html.parser")
    #find all the data-stat attributes and get the link the the player page of the gamelog
    player_elements = new_soup.find_all(attrs={"data-stat":"player"})
    base_url = 'https://www.pro-football-reference.com'
    for player in player_elements:
        player_a = player.find("a")
        if player_a:
            player_list +=[base_url + player_a["href"] + '/gamelog/']
    return player_list

In [5]:
def get_player_data_frame(play_url):
    
    #Pull HTML tables into pandas
    player_all_df = pd.read_html(play_url)
    #Use First Table (Regular Season)
    player_df = player_all_df[0]
    #Combine Mult-Layer column names
    player_df.columns = [f'{i}|{j}' if 'Unnamed' not in str(i) else f'{j}' for i,j in player_df.columns]

    #Get html for player page to get player info
    player_page = requests.get(play_url)
    #Use Beautiful soup on the html
    player_soup = BeautifulSoup(player_page.content,'html.parser')
    #Find id meta for player info
    player_meta = player_soup.find(id="meta")
    #Get just the text
    player_desc =  player_meta.get_text()
    #Regex search for the player position
    player_position = re.search('Position:(.*)',player_desc).group(1)
    #Set player position column
    player_df['Position'] = player_position
    #Get just the text for the player name for file
    player_name = player_soup.h1.span.get_text()
    return player_df, player_position,player_name

In [6]:
base_path = fr'C:\Users\Tyler\OneDrive\Documents'

In [7]:
import os

time.sleep(5)
yr=2022
roster_list = get_team_urls(url,yr)
time.sleep(5)
for roster in roster_list:

    time.sleep(5)
    player_list = get_player_urls(roster)

    for player in player_list:
        time.sleep(5)
        player_data, player_pos, player_name = get_player_data_frame(player)

        player_pos = player_pos.strip()

        newpath = fr'{base_path}\{yr}\{player_pos}'
        if not os.path.exists(newpath):
            os.makedirs(newpath)

        player_data.to_csv(fr"{newpath}\{player_name}.csv")

AttributeError: 'NoneType' object has no attribute 'group'